# Introduction to Chains in LangChain

A chain in LangChain is about putting stuff together. In the context of working with LLMs what is that stuff?

Well, that can be as simple as connecting a prompt with an LLM model.

## Model

To do that we need abstractions for each of these elements (the prompt and the LLM), so let's see how LangChain handles that.

In [2]:
from langchain.llms import OpenAI

llm = OpenAI()

llm.predict("Tell me a joke")

'\n\nQ: What did the fish say when it hit the wall?\nA: Dam!'

In [3]:
llm.invoke("Tell me a joke about a Panda and the Python programming language")

'\n\nQ: What did the Panda say when he saw the Python code?\nA: "Wow, that\'s a lot of Bamboozles!"'

Great! These are completion examples, let's look at interacting with the chat models (like ChatGPT and so on).

In [6]:
from langchain.chat_models import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-3.5-turbo-1106")

print(chat_model.predict("Hi"))
# or
chat_model.invoke("Hi")

Hello! How can I help you today?


AIMessage(content='Hello! How can I assist you today?')

Understanding diff between chatmodel output and llm model output

In [16]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

llm = OpenAI()
chat_model =  ChatOpenAI()

## Prompt

- advatange of abstracting partial variables that go into a prompt string which makes this format a better option with respect to just a regular string.

See: https://python.langchain.com/docs/modules/model_io/prompts

In [20]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is a good hobby to develop to be more {activity}?")

prompt.format(activity="creative") 

'What is a good hobby to develop to be more creative?'

ChatPromptTemplate

In [25]:
from langchain.prompts import ChatPromptTemplate

template_prompt = "You are a helpful assistant that will translate {input_lang} to {output_lang}."
human_prompt = "{input_text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template_prompt), 
    ("human", human_prompt)
])

chat_prompt.format_messages(input_lang="English", output_lang="Portuguese", input_text="Hi! I am learning the 'langchain' framework")

[SystemMessage(content='You are a helpful assistant that will translate English to Portuguese.'),
 HumanMessage(content="Hi! I am learning the 'langchain' framework")]

## Output Parser

[source](https://python.langchain.com/docs/get_started/quickstart#:~:text=for%20more%20detail.-,output%20parsers) 

3 Types

- LLM text into structured information (e.g JSON)
- ChatMessage into string
- Extra info (e.g. OpenAI function invocation) into string

In [26]:
from langchain.schema import BaseOutputParser
from typing import List


class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""
    def parse(self, text: str) -> List[str]:
        return text.strip().split(",")
    
    
CommaSeparatedListOutputParser().parse("I, love, learning, new, and, cool, frameworks")

['I', ' love', ' learning', ' new', ' and', ' cool', ' frameworks']

## Chains - Putting Everything Together

Ok, now that we have these basic elements. Let's put them together into a [chain](https://python.langchain.com/docs/get_started/introduction).

Composing with [LCEL](https://python.langchain.com/docs/expression_language)

In [7]:
from typing import List

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser[List[str]]):
    """Parse the output of an LLM call to a comma-separated list."""


    def parse(self, text: str) -> List[str]:
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

template = """You are a helpful assistant who generates comma separated lists.
A user will pass in a category, and you should generate 5 objects in that category in a comma separated list.
ONLY return a comma separated list, and nothing more."""
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])
chain = chat_prompt | ChatOpenAI() | CommaSeparatedListOutputParser()
chain.invoke({"text": "colors"})
# >> ['red', 'blue', 'green', 'yellow', 'orange']

['red', 'blue', 'green', 'yellow', 'purple']

Multiple Chains

In [1]:
from operator import itemgetter

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

prompt1 = ChatPromptTemplate.from_template("what is the city {person} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what country is the city {city} in? respond in {language}"
)

model = ChatOpenAI()

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"city": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

chain2.invoke({"person": "obama", "language": "spanish"})

'El país en el que nació la ciudad de Honolulu, Hawái, donde nació Barack Obama, el 44º presidente de los Estados Unidos, es Estados Unidos.'

As we can see from these examples, a chain in Langchain is nothing more than just a building block for putting things together.

This capability allows you build extremely interesting and complex functionality out of simple objects like prompt templates and output parsers.


However, for that to work well there is a need for some standard interface to allow this process of composition to run seamlessly. That's where [LangChain's LCEL language](https://python.langchain.com/docs/get_started/introduction#:~:text=JavaScript%20LangChain%20library.-,langchain%20expression%20language%20(lcel)) comes into play. We'll look at that in the next notebook lesson.

